
# Ejercicios Práctico Clase 07

## Actividad 1

**Enunciado.** Usar `groupby()` de Pandas para segmentar el dataset y **calcular el total de ventas por producto**, presentando el resultado de forma clara y concisa.

**Datos disponibles.** Columnas: `producto`, `mes`, `ventas`.  

```python
data = {
  'producto': ['A','B','A','C','B','C','A','B','C'],
  'mes':      ['Enero','Enero','Febrero','Febrero','Marzo','Marzo','Marzo','Enero','Febrero'],
  'ventas':   [150, 200, 250, 300, 100, 400, 350, 200, 300]
}
````

## Pasos para resolverlo (no todos son indispensables)

1. Crear un `DataFrame` con `data` y confirmar estructura mínima (filas/columnas y tipos).
2. Agregar por producto: `df.groupby('producto')['ventas'].sum()`.
3. Convertir el resultado a `DataFrame` con `reset_index()` y renombrar la métrica a `ventas_totales`.
4. Ordenar de mayor a menor para una lectura rápida.
5. Mostrar en pantalla la tabla final (quedará con dos columnas: `producto` y `ventas_totales`).

In [6]:
# =====================================================
# Actividad 1: Agrupar y sumar ventas por producto
# =====================================================

import pandas as pd

# 1) Cargamos el conjunto de datos en un DataFrame
data = {
    'producto': ['A', 'B', 'A', 'C', 'B', 'C', 'A', 'B', 'C'],
    'mes':      ['Enero', 'Enero', 'Febrero', 'Febrero', 'Marzo', 'Marzo',
                 'Marzo', 'Enero', 'Febrero'],
    'ventas':   [150, 200, 250, 300, 100, 400, 350, 200, 300]
}
df = pd.DataFrame(data)

# 2) Agrupamos por producto y sumamos las ventas de cada grupo
df_totales = (
    df.groupby('producto', as_index=False)
      .agg(ventas_totales=('ventas', 'sum'))
)

# 3) Ordenar de mayor a menor para lectura rápida (opcional)
df_totales = df_totales.sort_values('ventas_totales', ascending=False)

# 4) Mostrar resultados de forma clara
print("Ventas totales por producto:")
display(df_totales)


Ventas totales por producto:


,producto,ventas_totales
2,C,1000
0,A,750
1,B,500


# Actividad 2 · Tablas dinámicas (pivot_table) para resumir ventas

**Enunciado.** Usar `pivot_table()` de Pandas para crear una **tabla dinámica** que muestre las **ventas mensuales por producto**, con **productos en filas**, **meses en columnas** y **ventas totales** como valores. Preparar el resultado para una presentación clara.

## Pasos posibles para la resolución
1. **Cargar** el archivo `ventas.csv` en un `DataFrame`.
2. **Ordenar** los meses (opcional pero recomendado) convirtiendo la columna `mes` en categórica con el orden Enero a Diciembre.
3. **Construir** la tabla dinámica con:
   - `index="producto"`
   - `columns="mes"`
   - `values="ventas"`
   - `aggfunc="sum"`
   - `fill_value=0` para celdas sin datos.
   - `margins=True, margins_name="Total"` para totales por fila y columna.
4. **Reordenar** las columnas según el orden de meses y, al final, la columna **Total** si existe.
5. **Formatear** la salida (p. ej., sin decimales) y **mostrar** la tabla final, lista para incluir en una presentación.



In [11]:
# =====================================================
# Actividad 2: Tabla dinámica (pivot_table)
# de las ventas mensuales por producto
# =====================================================

import pandas as pd

# 1) Cargamos el conjunto de datos en un DataFrame
data = {
    'producto': ['A', 'B', 'A', 'C', 'B', 'C', 'A', 'B', 'C'],
    'mes':      ['Enero', 'Enero', 'Febrero', 'Febrero', 'Marzo',
                 'Marzo', 'Marzo', 'Enero', 'Febrero'],
    'ventas':   [150, 200, 250, 300, 100, 400, 350, 200, 300]
}
df = pd.DataFrame(data)

# 2) (Opcional) Asegurar orden lógico de los meses en columnas
# (Ver nota al final)
meses_orden = ["Enero","Febrero","Marzo","Abril","Mayo","Junio",
               "Julio","Agosto","Septiembre","Octubre","Noviembre","Diciembre"]
if "mes" in df.columns:
    df["mes"] = pd.Categorical(df["mes"], categories=meses_orden, ordered=True)

# 3) Crear tabla dinámica: filas=producto, columnas=mes, valores=ventas (sumadas)
tabla = pd.pivot_table(
    df,
    index="producto",
    columns="mes",
    values="ventas",
    aggfunc="sum",  # Sumamos..
    fill_value=0,   # celdas sin datos -> 0
    margins=True,   # agrega totales por fila/columna
    margins_name="Total" # Nombre columna...
)

# 4) Reordenar columnas según el orden de meses
# (si no están todas, toma las presentes)
cols_ordenadas = [m for m in meses_orden if m in tabla.columns] + (["Total"] if "Total" in tabla.columns else [])
tabla = tabla.reindex(columns=cols_ordenadas)

# 5) Mostrar resultados formateados para presentación
pd.set_option("display.float_format", "{:,.0f}".format)  # sin decimales
print("Ventas mensuales por producto (suma):")
display(tabla)


Ventas mensuales por producto (suma):


mes,Enero,Febrero,Marzo,Total
producto,,,,
A,150,250,350,750
B,400,0,100,500
C,0,600,400,1000
Total,550,850,850,2250


**Nota Paso 2:** Esa línea primero verifica que exista la columna `mes` y, si existe, la **convierte al tipo categórico** de Pandas fijando **las únicas categorías válidas** y **su orden** según `meses_orden` (Enero a Diciembre) y marcando que ese orden es el que se debe respetar (`ordered=True`).

Con esto, cualquier **ordenamiento, groupby o pivot** usará el **orden cronológico** de los meses en lugar del alfabético, y cualquier valor que no esté en la lista de categorías quedará como **NaN** (útil para detectar nombres de mes mal escritos).


**Nota sobre el "FutureWarning":** Ese `FutureWarning` avisa que en próximas versiones de Pandas cambiará el valor por defecto del parámetro observed en operaciones de agrupamiento con columnas categóricas (como el mes convertido a `Categorical`).

Hoy el default es `observed=False` (incluye todas las categorías posibles, aunque no aparezcan en los datos) y va a pasar a `observed=True` (mostrará solo las categorías realmente presentes en el cruce).

**Nota Paso 4:** Crea una **lista de columnas ordenadas** según `meses_orden`, pero **solo con los meses que realmente existen** en `tabla` (`if m in tabla.columns`). Al final, **agrega "Total"** si esa columna está presente.

Luego `reindex(columns=cols_ordenadas)` **reordena las columnas** de la tabla con esa lista, dejando los meses en orden cronológico y “Total” al final, sin fallar si falta algún mes.
